In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-11 04:53:12--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.67MB/s    in 0.2s    

2022-08-11 04:53:13 (4.67 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://amazonvine-bucket.s3.us-west-1.amazonaws.com/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Create DataFrames to match tables

In [28]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.functions import desc
# Read in the Review dataset as a DataFrame

In [29]:
# Create the vine_table. DataFrame
vine_df = df.select(["product_title", "review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------------+--------------+-----------+-------------+-----------+----+-----------------+
|       product_title|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------------+--------------+-----------+-------------+-----------+----+-----------------+
|Thrustmaster T-Fl...| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
|Tonsee 6 buttons ...| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|Hidden Mysteries:...|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|GelTabz Performan...|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|Zero Suit Samus a...|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|Psyclone Recharge...|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|Protection for yo...|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|   Nerf 3

In [41]:
# Create the vine_table 5 star_rating DataFrame
vine_df = df.select(["product_title", "review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_5stars_votes_df = vine_df.filter("star_rating == 5")
vine_5stars_votes_df.show()

+--------------------+--------------+-----------+-------------+-----------+----+-----------------+
|       product_title|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------------+--------------+-----------+-------------+-----------+----+-----------------+
|Thrustmaster T-Fl...| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
|Tonsee 6 buttons ...| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|Protection for yo...|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|   Nerf 3DS XL Armor|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
|One Piece: Pirate...| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
|Metal Arms: Glitc...|R1VTIA3JTYBY02|          5|            0|          0|   N|                N|
|Xbox 360 Media Re...| RIV24EQAIXA4O|          5|            0|          0|   N|                Y|
|Super Mar

In [49]:
# Sort 5 stars review by most helpful and voted
vine_5stars_votes_df.orderBy(col("total_votes").desc(),col("helpful_votes").desc()).show(truncate=False)

+--------------------------------------------------------------------+--------------+-----------+-------------+-----------+----+-----------------+
|product_title                                                       |review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------------------------------------------------------------+--------------+-----------+-------------+-----------+----+-----------------+
|Diablo III - PC/Mac                                                 |R2E1992AEAUA8Z|5          |4105         |4649       |N   |N                |
|Nintendo Wii Console                                                |R1CY66TNX7N4RX|5          |3873         |4143       |N   |N                |
|PlayStation Headsets                                                |R1875IL2IWLFSZ|5          |3505         |3658       |N   |Y                |
|SimCity - Limited Edition                                           |R3BQ88T77DIPNS|5          |2567         |3029   

In [56]:
# Count number of reviews
vine_df.count()

1785997

In [57]:
vine_5stars_votes_df.count()

1026924

In [60]:
# Deliverable 2 - Create a DataFrame or table where there are 20 or more total votes
vine_df_morethan20votes = vine_df.filter("total_votes>=20")
vine_df_morethan20votes.orderBy(col("total_votes").desc()).show(truncate=False)

+---------------------------------------+--------------+-----------+-------------+-----------+----+-----------------+
|product_title                          |review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------------------------------------+--------------+-----------+-------------+-----------+----+-----------------+
|SimCity - Limited Edition              |R3HI7AMO8MJ9PF|1          |10498        |10780      |N   |N                |
|Spore                                  |RK9RKIUMYF757 |1          |8650         |9459       |N   |N                |
|Assassin's Creed Unity                 |RTDOWG8IG081J |1          |5141         |5334       |N   |N                |
|Diablo III - PC/Mac                    |R2E1992AEAUA8Z|5          |4105         |4649       |N   |N                |
|Nintendo Wii Console                   |R1CY66TNX7N4RX|5          |3873         |4143       |N   |N                |
|Sony Playstation 3 160GB System        |R234OYV0YWWO93|

In [61]:
# Deliverable 2 - Create a DataFrame or table where the percentage of helpful_votes is equal to or greater than 50%
vine_df_helpful = vine_df_morethan20votes.filter((vine_df_morethan20votes["helpful_votes"])/(vine_df_morethan20votes["total_votes"]) >= 0.50)
vine_df_helpful.show()

+--------------------+--------------+-----------+-------------+-----------+----+-----------------+
|       product_title|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------------+--------------+-----------+-------------+-----------+----+-----------------+
|Valve - DOTA 2 St...| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|ONE PIECE Pirate ...|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|       Madden NFL 16|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|Infinity 3.0 Star...|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|       Madden NFL 16|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|Infinity 3.0 Star...|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
|Xbox One 500GB Co...| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|Disney In

In [62]:
# Deliverable 2 - Create a DataFrame or table where there is a Vine Review
vine_df_yes = vine_df_helpful.filter(vine_df_helpful["vine"] == "Y")
vine_df_yes.show()

+--------------------+--------------+-----------+-------------+-----------+----+-----------------+
|       product_title|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------------+--------------+-----------+-------------+-----------+----+-----------------+
|Turtle Beach - Ea...|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|PDP AG7 True Wire...|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
|Turtle Beach - Ea...| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
|Antec X-1 Cooler ...| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
|Turtle Beach - Ea...|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
|Turtle Beach Ear ...|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|
|Turtle Beach - Ea...|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
|Turtle Be

In [63]:
# Deliverable 2 - Create a DataFrame or table where there isn't a Vine Review
vine_df_no = vine_df_helpful.filter(vine_df_helpful["vine"] == "N")
vine_df_no.show()

+--------------------+--------------+-----------+-------------+-----------+----+-----------------+
|       product_title|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------------+--------------+-----------+-------------+-----------+----+-----------------+
|Valve - DOTA 2 St...| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|ONE PIECE Pirate ...|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|       Madden NFL 16|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|Infinity 3.0 Star...|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|       Madden NFL 16|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|Infinity 3.0 Star...|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
|Xbox One 500GB Co...| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|Disney In

In [64]:
# Deliverable 2 - The total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews are calculated for all Vine and non-Vine reviews
vine_yes_total = vine_df_yes.count()
vine_yes_fivestar = vine_df_yes.filter("star_rating=5").count()
vine_yes_percentage = (vine_yes_fivestar / vine_yes_total) * 100

vine_no_total = vine_df_no.count()
vine_no_fivestar = vine_df_no.filter("star_rating=5").count()
vine_no_percentage = (vine_no_fivestar / vine_no_total) * 100

summary_df = spark.createDataFrame([("Yes", vine_yes_total, vine_yes_fivestar, vine_yes_percentage), ("No", vine_no_total, vine_no_fivestar, vine_no_percentage)], ["Vine Review", "Total Reviews", "Five Star Reviews", "% Five Star"])

summary_df.show()

+-----------+-------------+-----------------+------------------+
|Vine Review|Total Reviews|Five Star Reviews|       % Five Star|
+-----------+-------------+-----------------+------------------+
|        Yes|           94|               48| 51.06382978723404|
|         No|        40471|            15663|38.701786464381904|
+-----------+-------------+-----------------+------------------+



### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
## Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazonvinedatabase-1.c4tt58msfrhj.us-west-1.rds.amazonaws.com:5432/amazonvinedatabase-1"
config = {"user":"postgres", 
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)